In [1]:
import numpy as np

In [2]:
from iam.features_and_letters_only import IAM, Layer, load_corpus, alphabet, Connection

Load the words

In [3]:
corpus = load_corpus()
print(corpus[:5])

n_words = len(corpus)
print(n_words)

     word  frequency
40   abel         20
68   able        216
322  acid         13
359  acre          9
395  acts         39
1188


Letter-word excitation
Letter-word inhibition
Word-word inhibition
Word-letter excitation
.07
.04
.21
.30

In [4]:
class IAMWithWords1(IAM):
    def __init__(self):
        super().__init__()
        
        self.word_layer = Layer(
            shape=(1188, ),
            resting_activation=0,  # it will be a random number between -0.05 and 0
            minimum_activation=self.m,
            maximum_activation=self.M,
            decay_rate=self.theta
        )
        self._layers.append(self.word_layer)
        
        # Letter-to-word connections
        is_excitatory = np.array([[[word[position].lower() == letter.lower()
                                    for word in corpus.word]
                                   for letter in alphabet]
                                  for position in range(4)])
        is_excitatory = np.reshape(is_excitatory, newshape=(-1, is_excitatory.shape[-1]))
        # For one letter
        letter_to_word_excitatory = 0.07
        letter_to_word_inhibitory = 0.04
        letter_to_word_weights = np.where(
            is_excitatory,
            letter_to_word_excitatory,
            - letter_to_word_inhibitory
        )        
        letter_to_word_connection = Connection(
            layer_from=self.letter_layer,
            layer_to=self.word_layer,
            weights=letter_to_word_weights
        )

In [5]:
iam_with_words_1 = IAMWithWords1()
iam_with_words_1.present_word('WORK')

In [6]:
iam_with_words_1.run_cycle()
iam_with_words_1.print_active_letters()
(corpus
 .assign(activation=iam_with_words_1.word_layer.activations)
 .sort_values(by='activation', ascending=False)
)

letter 1: ['W']
letter 2: ['O' 'Q']
letter 3: ['R']
letter 4: ['K']


,word,frequency,activation
40,abel,20,0.0
30845,plug,23,0.0
30831,plot,37,0.0
30786,plea,11,0.0
30750,play,200,0.0
...,...,...,...
17141,glen,7,0.0
17090,glad,38,0.0
17073,give,391,0.0
17054,girl,220,0.0


In [7]:
iam_with_words_1.run_cycle()
iam_with_words_1.print_active_letters()
(corpus
 .assign(activation=iam_with_words_1.word_layer.activations)
 .sort_values(by='activation', ascending=False)
)

letter 1: ['W']
letter 2: ['O' 'Q']
letter 3: ['R']
letter 4: ['K']


,word,frequency,activation
45891,work,760,0.00720
45882,word,274,0.00445
45943,worn,23,0.00445
45890,wore,65,0.00445
9019,cork,9,0.00390
...,...,...,...
19190,hind,6,-0.00120
19206,hino,13,-0.00120
19209,hint,9,-0.00120
19219,hips,8,-0.00120


Why is "cork" more activated than "word"?

In [8]:
class IAMWithWords2(IAMWithWords1):
    def __init__(self):
        super().__init__()
        
        # Word-to-word connections
        word_to_word_inhibitory = 0.21
        word_to_word_weights = np.where(
            np.identity(n_words),
            0,
            -word_to_word_inhibitory
        )
        word_to_word_connection = Connection(
            layer_from=self.word_layer,
            layer_to=self.word_layer,
            weights=word_to_word_weights
        )
        
        # Word-to-letter        
        is_excitatory = np.array([[[word[position].lower() == letter.lower()
                                    for word in corpus.word]
                                   for letter in alphabet]
                                  for position in range(4)])
        is_excitatory = np.reshape(is_excitatory, newshape=(-1, is_excitatory.shape[-1]))
        
        word_to_letter_excitatory = 0.30
        word_to_letter_weights = np.where(
            is_excitatory.T,
            word_to_letter_excitatory,
            0  # There are no inhibitory connections from the word layer to the letter layer
        )        
        word_to_letter_connection = Connection(
            layer_from=self.word_layer,
            layer_to=self.letter_layer,
            weights=word_to_letter_weights
        )

In [9]:
iam_with_words_2 = IAMWithWords2()
iam_with_words_2.present_word('WORK')

In [10]:
# Cycle 1
iam_with_words_2.run_cycle()
iam_with_words_2.print_active_letters()
(corpus
 .assign(activation=iam_with_words_2.word_layer.activations)
 .sort_values(by='activation', ascending=False)
)

letter 1: ['W']
letter 2: ['O' 'Q']
letter 3: ['R']
letter 4: ['K']


,word,frequency,activation
40,abel,20,0.0
30845,plug,23,0.0
30831,plot,37,0.0
30786,plea,11,0.0
30750,play,200,0.0
...,...,...,...
17141,glen,7,0.0
17090,glad,38,0.0
17073,give,391,0.0
17054,girl,220,0.0


In [11]:
# Cycle 2
iam_with_words_2.run_cycle()
iam_with_words_2.print_active_letters()
(corpus
 .assign(activation=iam_with_words_2.word_layer.activations)
 .sort_values(by='activation', ascending=False)
)

letter 1: ['W']
letter 2: ['O' 'Q']
letter 3: ['R']
letter 4: ['K']


,word,frequency,activation
45891,work,760,0.00720
45882,word,274,0.00445
45943,worn,23,0.00445
45890,wore,65,0.00445
9019,cork,9,0.00390
...,...,...,...
19190,hind,6,-0.00120
19206,hino,13,-0.00120
19209,hint,9,-0.00120
19219,hips,8,-0.00120


The above is the same as without connection from the word layer because the word layer has just been activated.

In [12]:
# Cycle 3
iam_with_words_2.run_cycle()
iam_with_words_2.print_active_letters()
(corpus
 .assign(activation=iam_with_words_2.word_layer.activations)
 .sort_values(by='activation', ascending=False)
)

letter 1: ['W']
letter 2: ['O' 'Q']
letter 3: ['R']
letter 4: ['K']


,word,frequency,activation
45891,work,760,0.006346
45882,word,274,0.002642
45890,wore,65,0.002642
45943,worn,23,0.002642
9019,cork,9,0.001908
...,...,...,...
19190,hind,6,-0.006559
19206,hino,13,-0.006559
19209,hint,9,-0.006559
19219,hips,8,-0.006559


All the words have been inhibited somewhat, "work" - less so. Let's run 10  more cycles.

In [13]:
iam_with_words_2.run_n_cycles(10)
iam_with_words_2.print_active_letters()
(corpus
 .assign(activation=iam_with_words_2.word_layer.activations)
 .sort_values(by='activation', ascending=False)
)

letter 1: ['W']
letter 2: ['O' 'Q']
letter 3: ['R']
letter 4: ['K']


,word,frequency,activation
45891,work,760,0.294191
45943,worn,23,0.022595
45882,word,274,0.022595
45890,wore,65,0.022595
46244,york,301,0.001479
...,...,...,...
19190,hind,6,-0.094779
19206,hino,13,-0.094779
19209,hint,9,-0.094779
19219,hips,8,-0.094779


Now, "work" is much more activated than the other words.

In [14]:
iam_with_words_2.run_n_cycles(10)
iam_with_words_2.print_active_letters()
(corpus
 .assign(activation=iam_with_words_2.word_layer.activations)
 .sort_values(by='activation', ascending=False)
)

letter 1: ['W']
letter 2: ['O' 'Q']
letter 3: ['R']
letter 4: ['K']


,word,frequency,activation
45891,work,760,0.668042
45943,worn,23,-0.008980
45882,word,274,-0.008980
45890,wore,65,-0.008980
9019,cork,9,-0.026162
...,...,...,...
19190,hind,6,-0.151383
19206,hino,13,-0.151383
19209,hint,9,-0.151383
19219,hips,8,-0.151383


Now, "work" is the only active word.

*Exercise*: set word-to-word inhibition to zero. Run the model until you see an overflow warning. Find out why you get this warning.

Let's check that the code we've moved to the `iam` module behaves in the same way.

In [15]:
from iam import IAM as IAMTest

In [16]:
iam_with_words_2_test = IAMWithWords2()
iam_with_words_2_test.present_word('WORK')
iam_test = IAMTest()
iam_test.present_word('WORK')

In [17]:
# Test that activations are identical.
iam_test.run_cycle()
iam_with_words_2_test.run_cycle()

In [18]:
def check_equal():
    return all([
        np.array_equal(layer_A.activations, layer_B.activations)
        for layer_A, layer_B in zip(iam_test.layers, iam_with_words_2_test.layers)
    ])

In [19]:
check_equal()

True

In [21]:
identical_activations = list()
for _ in range(50):
    iam_test.run_cycle()
    iam_with_words_2_test.run_cycle()
    identical_activations.append(check_equal())
    
print(all(identical_activations))

True
